This notebook is based on [an official 🤗 notebook - "How to fine-tune a model on text classification"](https://github.com/huggingface/notebooks/blob/6ca682955173cc9d36ffa431ddda505a048cbe80/examples/text_classification.ipynb). The main aim of this notebook is to show the process of conversion from vanilla 🤗 to [Ray AIR](https://docs.ray.io/en/latest/ray-air/getting-started.html) 🤗 without changing the training logic unless necessary.

Uncomment and run the following line in order to install all the necessary dependencies:

In [ ]:
#! pip install "datasets" "transformers>=4.19.0" "torch>=1.10.0" "mlflow" "ray[tune, data]>=1.13"

# Set up Ray

We will use `ray.init()` to initialize a local cluster. By default, this cluster will be compromised of only the machine you are running this notebook on. You can also run this notebook on an Anyscale cluster.

If you would like to use [Ray Client](https://docs.ray.io/en/latest/cluster/ray-client.html) and connect to an external cluster, please see this example (TODO). This notebook *will not* run in Ray Client mode.

In [2]:
from pprint import pprint
import ray

ray.init()

RayContext(dashboard_url='', python_version='3.7.13', ray_version='2.0.0.dev0', ray_commit='e2ee2140f97ca08b70fd0f7561038b7f8d958d63', address_info={'node_ip_address': '172.28.0.2', 'raylet_ip_address': '172.28.0.2', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-05-12_18-30-10_467499_75/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-05-12_18-30-10_467499_75/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-05-12_18-30-10_467499_75', 'metrics_export_port': 64840, 'gcs_address': '172.28.0.2:58661', 'address': '172.28.0.2:58661', 'node_id': '65d091b8f504ccd72024fd0b1a8445a8f9ea43e86bcbf67868c22ba7'})

We can check the resources our cluster is composed of. If you are running this notebook on your local machine or Google Colab, you should see the number of CPU cores and GPUs available on the said machine.

In [3]:
pprint(ray.cluster_resources())

{'CPU': 2.0,
 'GPU': 1.0,
 'accelerator_type:T4': 1.0,
 'memory': 7855477556.0,
 'node:172.28.0.2': 1.0,
 'object_store_memory': 3927738777.0}


In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/). We will be running the training using [Ray AIR](https://docs.ray.io/en/latest/ray-air/getting-started.html).

You can change those two variables to control whether the training (which we will get to later) uses CPUs or GPUs, and how many workers should be spawned. Each worker will claim one CPU or GPU. Make sure not to request more resources than the resources present!

By default, we will run the training with one GPU worker.

In [4]:
use_gpu = True
num_workers = 1

# Fine-tuning a model on a text classification task

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences. If you would like to learn more, refer to the [original notebook](https://github.com/huggingface/notebooks/blob/6ca682955173cc9d36ffa431ddda505a048cbe80/examples/text_classification.ipynb).

Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (so same training set as `mnli` but different validation and test sets):

In [5]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [6]:
task = "cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.

Apart from `mnli-mm` being a special code, we can directly pass our task name to those functions.

As Ray AIR doesn't provide integrations for 🤗 Datasets yet, we will simply run the normal 🤗 Datasets code to load the dataset from the Hub.

In [7]:
from datasets import load_dataset

actual_task = "mnli" if task == "mnli-mm" else task
datasets = load_dataset("glue", actual_task)

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

We will also need the metric. In order to avoid serialization errors, we will load the metric inside the training workers later. Therefore, now we will just define the function we will use.

In [8]:
from datasets import load_metric

def load_metric_fn():
    return load_metric('glue', actual_task)

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric).

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [10]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can them write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [11]:
def preprocess_function(examples, *, tokenizer):
    sentence1_key, sentence2_key = task_to_keys[task]
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [12]:
encoded_datasets = datasets.map(preprocess_function, batched=True, fn_kwargs=dict(tokenizer=tokenizer))

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

For Ray AIR, instead of using 🤗 Dataset objects directly, we will convert them to [Ray Datasets](https://docs.ray.io/en/latest/data/dataset.html). Both are backed by Arrow tables, so the conversion is straightforward. As we will use the built-in `ray.data.from_huggingface` function, we import `ray.data`.

In [13]:
import ray.data

ray_datasets = ray.data.from_huggingface(encoded_datasets)

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it.

Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class.

We will not go into details about each specific component of the training (see the [original notebook](https://github.com/huggingface/notebooks/blob/6ca682955173cc9d36ffa431ddda505a048cbe80/examples/text_classification.ipynb) for that). The tokenizer is the same as we have used to encoded the dataset before.

The main difference when using the Ray AIR is that we need to create our 🤗 Transformers `Trainer` inside a function (`trainer_init_per_worker`) and return it. That function will be passed to the `HuggingFaceTrainer` and ran on every Ray worker. The training will then proceed by the means of PyTorch DDP.

Make sure that you initialize the model, metric and tokenizer inside that function. Otherwise, you may run into serialization errors.

Please note that if you don't want to use CUDA, you need to explicitly set `no_cuda=True` inside the `TrainingArguments`. Here, we use the value of the `use_gpu` variable we set in the beginning.

Furthermore, `push_to_hub=True` is not yet supported. Ray will however checkpoint the model at every epoch, allowing you to push it to hub manually. We will do that after the training.

If you wish to use thrid party logging libraries, such as MLFlow or Weights&Biases, do not set them in `TrainingArguments` (they will be automatically disabled) - instead, you should be passing Ray AIR callbacks to `HuggingFaceTrainer`'s `run_config`. In this example, we will use MLFlow.

We also set `disable_tqdm=True` to declutter the output a little.

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import torch

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
name = f"{model_name}-finetuned-{task}"

def trainer_init_per_worker(train_dataset, eval_dataset = None, **config):
    print(f"Is CUDA available: {torch.cuda.is_available()}")
    metric = load_metric_fn()
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
    args = TrainingArguments(
        name,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=5,
        weight_decay=0.01,
        push_to_hub=False,
        disable_tqdm=True,
        no_cuda=not use_gpu,
    )

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        if task != "stsb":
            predictions = np.argmax(predictions, axis=1)
        else:
            predictions = predictions[:, 0]
        return metric.compute(predictions=predictions, references=labels)

    trainer = Trainer(
        model,
        args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    print("Starting training")
    return trainer

With our `trainer_init_per_worker` complete, we can now instantiate the `HuggingFaceTrainer`. Aside from the function, we set the `scaling_config`, controlling the amount of workers and resources used, and the `datasets` we will use for training and evaluation.

We specify the `MlflowLoggerCallback` inside the `run_config`.

In [15]:
from ray.ml.train.integrations.huggingface import HuggingFaceTrainer
from ray.ml import RunConfig
from ray.tune.integration.mlflow import MLflowLoggerCallback

trainer = HuggingFaceTrainer(
    trainer_init_per_worker=trainer_init_per_worker,
    scaling_config={"num_workers": num_workers, "use_gpu": use_gpu},
    datasets={"train": ray_datasets["train"], "evaluation": ray_datasets[validation_key]},
    run_config=RunConfig(callbacks=[MLflowLoggerCallback(experiment_name=name)])
)

Finally, we call the `fit` method to being training with Ray AIR. We will save the `Result` object to a variable so we can access metrics and checkpoints.

In [16]:
result = trainer.fit()

Trial name,status,loc,iter,total time (s),loss,learning_rate,epoch
HuggingFaceTrainer_bb9dd_00000,TERMINATED,172.28.0.2:419,5,222.391,0.1575,1.30841e-06,5


(BaseWorkerMixin pid=455) 2022-05-12 18:31:33,158	INFO torch.py:347 -- Setting up process group for: env:// [rank=0, world_size=1]


(BaseWorkerMixin pid=455) Is CUDA available: True


Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]
Downloading:   2%|▏         | 5.63M/256M [00:00<00:04, 59.1MB/s]
Downloading:   5%|▍         | 12.2M/256M [00:00<00:03, 65.0MB/s]
Downloading:   7%|▋         | 18.5M/256M [00:00<00:03, 65.6MB/s]
Downloading:  10%|▉         | 25.3M/256M [00:00<00:03, 67.5MB/s]
Downloading:  12%|█▏        | 31.7M/256M [00:00<00:03, 66.6MB/s]
Downloading:  15%|█▌        | 38.3M/256M [00:00<00:03, 67.6MB/s]
Downloading:  18%|█▊        | 44.8M/256M [00:00<00:03, 67.6MB/s]
Downloading:  20%|██        | 51.2M/256M [00:00<00:03, 66.6MB/s]
Downloading:  23%|██▎       | 57.9M/256M [00:00<00:03, 67.5MB/s]
Downloading:  25%|██▌       | 64.7M/256M [00:01<00:02, 68.6MB/s]
Downloading:  28%|██▊       | 71.2M/256M [00:01<00:02, 66.6MB/s]
Downloading:  31%|███       | 78.0M/256M [00:01<00:02, 67.9MB/s]
Downloading:  33%|███▎      | 84.5M/256M [00:01<00:02, 68.0MB/s]
Downloading:  36%|███▌      | 91.1M/256M [00:01<00:02, 68.2MB/s]
Downloading:  38%|███▊      | 97.7

(BaseWorkerMixin pid=455) Starting training


...

(BaseWorkerMixin pid=455) Saving model checkpoint to distilbert-base-uncased-finetuned-cola/checkpoint-2675
(BaseWorkerMixin pid=455) Configuration saved in distilbert-base-uncased-finetuned-cola/checkpoint-2675/config.json
(BaseWorkerMixin pid=455) Model weights saved in distilbert-base-uncased-finetuned-cola/checkpoint-2675/pytorch_model.bin
(BaseWorkerMixin pid=455) tokenizer config file saved in distilbert-base-uncased-finetuned-cola/checkpoint-2675/tokenizer_config.json
(BaseWorkerMixin pid=455) Special tokens file saved in distilbert-base-uncased-finetuned-cola/checkpoint-2675/special_tokens_map.json
(BaseWorkerMixin pid=455) ***** Running Evaluation *****
(BaseWorkerMixin pid=455)   Num examples = 1043
(BaseWorkerMixin pid=455)   Batch size = 16
(BaseWorkerMixin pid=455) The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `Distil

(BaseWorkerMixin pid=455) {'eval_loss': 0.8616615533828735, 'eval_matthews_correlation': 0.5420036503219092, 'eval_runtime': 1.2577, 'eval_samples_per_second': 829.302, 'eval_steps_per_second': 52.477, 'epoch': 5.0}


(BaseWorkerMixin pid=455) Model weights saved in distilbert-base-uncased-finetuned-cola/checkpoint-2675/pytorch_model.bin
(BaseWorkerMixin pid=455) tokenizer config file saved in distilbert-base-uncased-finetuned-cola/checkpoint-2675/tokenizer_config.json
(BaseWorkerMixin pid=455) Special tokens file saved in distilbert-base-uncased-finetuned-cola/checkpoint-2675/special_tokens_map.json
(BaseWorkerMixin pid=455) 
(BaseWorkerMixin pid=455) 
(BaseWorkerMixin pid=455) Training completed. Do not forget to share your model on huggingface.co/models =)
(BaseWorkerMixin pid=455) 
(BaseWorkerMixin pid=455) 


(BaseWorkerMixin pid=455) {'train_runtime': 187.8585, 'train_samples_per_second': 227.592, 'train_steps_per_second': 14.239, 'train_loss': 0.30010223103460865, 'epoch': 5.0}
Trial HuggingFaceTrainer_bb9dd_00000 reported loss=0.1575,learning_rate=1.308411214953271e-06,epoch=5.0,step=2675,eval_loss=0.8616615533828735,eval_matthews_correlation=0.5420036503219092,eval_runtime=1.2577,eval_samples_per_second=829.302,eval_steps_per_second=52.477,train_runtime=187.8585,train_samples_per_second=227.592,train_steps_per_second=14.239,train_loss=0.30010223103460865,_timestamp=1652380513,_time_this_iter_s=39.63672137260437,_training_iteration=5,should_checkpoint=True with parameters={}.
Trial HuggingFaceTrainer_bb9dd_00000 completed. Last result: loss=0.1575,learning_rate=1.308411214953271e-06,epoch=5.0,step=2675,eval_loss=0.8616615533828735,eval_matthews_correlation=0.5420036503219092,eval_runtime=1.2577,eval_samples_per_second=829.302,eval_steps_per_second=52.477,train_runtime=187.8585,train_samp

2022-05-12 18:35:14,803	INFO tune.py:753 -- Total run time: 228.34 seconds (228.07 seconds for the tuning loop).


You can use the returned `Result` object to access metrics and the Ray AIR `Checkpoint` associated with the last iteration.

In [17]:
result

Result(metrics={'loss': 0.1575, 'learning_rate': 1.308411214953271e-06, 'epoch': 5.0, 'step': 2675, 'eval_loss': 0.8616615533828735, 'eval_matthews_correlation': 0.5420036503219092, 'eval_runtime': 1.2577, 'eval_samples_per_second': 829.302, 'eval_steps_per_second': 52.477, 'train_runtime': 187.8585, 'train_samples_per_second': 227.592, 'train_steps_per_second': 14.239, 'train_loss': 0.30010223103460865, '_timestamp': 1652380513, '_time_this_iter_s': 39.63672137260437, '_training_iteration': 5, 'time_this_iter_s': 39.64510202407837, 'should_checkpoint': True, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 5, 'trial_id': 'bb9dd_00000', 'experiment_id': 'db0c5ea784a44980819bf5e1bfb72c04', 'date': '2022-05-12_18-35-13', 'timestamp': 1652380513, 'time_total_s': 222.39091277122498, 'pid': 419, 'hostname': 'e618da00601e', 'node_ip': '172.28.0.2', 'config': {}, 'time_since_restore': 222.39091277122498, 'timesteps_since_restore': 0, 'iterations_since_resto

You can now use the checkpoint to run prediction with `HuggingFacePredictor`, which wraps around [🤗 Pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines). In order to distribute prediction, we use `BatchPredictor`. While this is not necessary for the very small example we are using (you could use `HuggingFacePredictor` directly), it will scale well to a large dataset.

In [18]:
from ray.ml.predictors.integrations.huggingface import HuggingFacePredictor
from ray.ml.batch_predictor import BatchPredictor
import pandas as pd

sentences = ['Bill whistled past the house.',
  'The car honked its way down the road.',
  'Bill pushed Harry off the sofa.',
  'the kittens yawned awake and played.',
  'I demand that the more John eats, the more he pay.']
predictor = BatchPredictor.from_checkpoint(
    checkpoint=result.checkpoint,
    predictor_cls=HuggingFacePredictor,
    task="text-classification",
)
data = ray.data.from_pandas(pd.DataFrame(sentences, columns=["sentence"]))
prediction = predictor.predict(data)
prediction = prediction.to_pandas()
prediction

Map Progress (2 actors 1 pending): 100%|██████████| 1/1 [00:16<00:00, 16.63s/it]


,label,score
0,LABEL_1,0.998539
1,LABEL_1,0.997706
2,LABEL_1,0.998476
3,LABEL_1,0.998498
4,LABEL_0,0.533578


To be able to share your model with the community, there are a few more steps to follow.

We have conducted the training on the Ray cluster, but share the model from the local enviroment - this will allow us to easily authenticate.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
# !apt install git-lfs

Now, load the model and tokenizer locally, and recreate the `Trainer`:

In [ ]:
from ray.ml.train.integrations.huggingface import load_checkpoint

hf_trainer = load_checkpoint(
    checkpoint=result.checkpoint,
    model=AutoModelForSequenceClassification,
    tokenizer=AutoTokenizer
)

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
hf_trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("sgugger/my-awesome-model")
```